In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

import utils.extractors as extractors
import utils.millers as millers

load_dotenv()

2025-07-08 15:35:26.794032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752006926.804103   55062 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752006926.807210   55062 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752006926.816088   55062 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752006926.816096   55062 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752006926.816097   55062 computation_placer.cc:177] computation placer alr

True

In [17]:
df = extractors.fetch_sample_transactions(limit=1_000)

# Basic sanity check
assert 'description' in df.columns and 'category_name' in df.columns, "Missing required columns"

In [18]:
# Step 2: Generate embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

descriptions = df['description'].astype(str).tolist()

embeddings = model.encode(descriptions, convert_to_numpy=True)

/home/brendan/anaconda3/envs/finance-scripts/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
# Step 3: Initialize Chroma DB (local, in-memory or persistent)
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [23]:
collection = chroma_client.get_or_create_collection(name="transactions")

In [24]:
metadata = df.loc[:, ['category_name']].reset_index().to_dict(orient='records')
metadata[0:10]


[{'id': 560, 'category_name': 'Amazon Uncategorized'},
 {'id': 561, 'category_name': 'Other Income'},
 {'id': 562, 'category_name': 'Other Income'},
 {'id': 557, 'category_name': 'Other Income'},
 {'id': 558, 'category_name': 'Other Income'},
 {'id': 559, 'category_name': 'Other Income'},
 {'id': 555, 'category_name': 'Technology'},
 {'id': 556, 'category_name': 'Periodicals'},
 {'id': 554, 'category_name': 'Periodicals'},
 {'id': 549, 'category_name': 'Delivery'}]

In [25]:
# Step 4: Insert into Chroma
# Chroma requires ids, embeddings, and metadata
collection.add(
    documents=descriptions,
    embeddings=embeddings,
    metadatas=metadata,
    ids=[f"txn_{i}" for i in range(len(descriptions))]
)

print(f"✅ Added {len(descriptions)} transactions to Chroma DB.")

✅ Added 266 transactions to Chroma DB.


# Find Nearest Neighbors

In [46]:
query_text = df.iloc[22].description
query_text

'H-E-B #031'

In [53]:
query_embedding = model.encode([query_text], convert_to_numpy=True)

results = collection.query(
    query_embeddings=query_embedding,
    n_results=5
)

for doc, meta, score in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
    print(f"→ Match: {doc:25} | Category: {meta['category_name']:25} | Score: {score:.4f}")

→ Match: H-E-B #031                | Category: Groceries                 | Score: 0.0000
→ Match: H-E-B  #269               | Category: Groceries                 | Score: 0.4760
→ Match: H-E-B  #269               | Category: Groceries                 | Score: 0.4760
→ Match: H-E-B  #269               | Category: Groceries                 | Score: 0.4760
→ Match: H-E-B  #269               | Category: Groceries                 | Score: 0.4760
